---
title: Scopes and Global Variables
order: 1
---

In this article we will discuss two python keywords - `global` and `nonlocal` - and scoping. Scoping can be especially hard and I cannot rule out that there are no errors in this text. For example, I wasn't able to find an official definition of a scope even in The Python Language Reference. So as always, this text is a mix of about how I understand the topic and what I was able to find and verify (no ChatGPT-only answers).

## Terminology

A scope is a block of code in which variables can be visible. A namespace is a mapping that defines variables. Each scope has its own namespace.

A global scope is the one at the level of a module. That is also the level that your IPython shell is in when you start a new one. Any variable you define at this level is a *global variable*.

Inside this scope you can create an inner scope, for example by defining a function. And now things start to be interesting. Let's assume we are in the scope of this function. Now, the current scope is called *local* scope and the outer scope (which also happens to be the global scope in this case) is called *nonlocal* scope.

This terminology holds for any number of nested scopes: The current scope is *local*, the closest higher scope is *nonlocal* and the top-most scope is *global*.

## Types of Scopes

There is surprisingly one scope above global - the *built-in* scope. The full list of scopes in Python is this:

- built-in
- global
- nonlocal (recursive)
- local

This means that when a variable is referenced in a local scope, Python first looks for it in the local scope. If it isn't found there the nearest enclosing scope is searched (the nonlocal scope). If it is not found there then the next higher scope is searched and so on, until a global scope is reached. If a variable is not found in the global scope, the built-in scope is searched. If the variable is not even here, a NameError is raised.

## Nesting of Scopes

It is important to remind here that nested scopes are created by *defining* new functions (or classes etc., see below), NOT by calling them. For example, these are not nested scopes, the function `innerfn` isn't actually "inner" and the function `outerfn` isn't actually "outer". They are same-level functions, just called from one another. They do not create nested scopes.

In [ ]:
def innerfn():
    print(x)

def outerfn():
    x = 2
    innerfn()

x = 1
outerfn()

1


We can see it because when `innerfn` is called from within `outerfn`, the number $1$ is printed, and not $2$. This is what happened:

- When `innerfn` is called from within `outerfn`, the variable `x` is first searched for locally.
- It is not found there, so Python completely skips the calling scope and searches for `x` in the scope where `innerfn` was defined - the global scope in this case.
- `x` is found there so its value is used ($1$). The value $2$ is not used, because Python never searches for `x` in the scope from which `innerfn` is called.
- If `x` wasn't found in the global scope (where `innerfn` is defined), Python would look in the built-in scope.

By the way, any variable defined in a function scope is treated as local to the function. Which means this code throws an error (UnboundLocalError):

In [ ]:
x = 1
def fn():
    print(x)
    x = 2
fn()

At the first line of the function, we need to get the variable `x`. It is not yet defined locally, so it doesn't exist in the local namespace, but it is defined in the enclosing (nonlocal) scope so we should be able to use that value, right? Wrong. Python knows in advance that somewhere down the body of the function the variable `x` is defined. So it treats it as a local variable and only looks for it in the local namespace. But it is not found there (at the point of the `print` statement), so an error is thrown.

## Manipulating Variable Lookup

So far we have seen that a variable is iteratively searched for from the local scope to the built-in scope until it is found and the corresponding value is used. By the way, it means that variables defined in higher-level scopes are *read-only* in lower scopes. I mean, we can use the variables in the lower scopes, but we cannot change them. Any attempt to change a variable defined in a higher level would mean we change the variable only in the local level:

In [6]:
x = 1
def fn():
    print(x)  # We can read variables defined in higher-level scopes
fn()

1


In [8]:
x = 1
def fn():
    x = 2
print(x)  # But we cannot change their values, as any such change would be local only and would not propagate to the higher level.

1


The keywords `global` and `nonlocal` enable us to both change higher-level variables and to manipulate the variable lookup process. For example, the keyword `global` says that:

- The variable(s) following it are to be immediatelly searched for in the global namespace (skipping the local and all non-local namespaces)
- It is possible to change these variables in the global namespace.

It is basically a direct and connection to the global namespace (for that particular variable(s)).

The keyword `nonlocal` does the exact same thing with the exception that it connects to the non-local scope.

We can demonstrate this behavior like this. First, we define three nested functions and define a variable `x` in each of them but with a different value:

In [10]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            x = 3
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn3: x = 3
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 0


Then we introduce a single change: In the bottom-most scope, instead of defining `x`, we declare that we want to use the one defined in the global scope. We can see that indeed the value of `x` in the global scope is then used here in the bottom-most scope. Other scopes remained untouched.

In [17]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            global x
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn3: x = 0
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 0


We can even change the global variable from the inner scopes:

In [18]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            global x
            x = 3
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn3: x = 3
inside fn2: x = 2
inside fn1: x = 1
inside global scope: x = 3


Then we see the same behavior is achieved by the keyword `nonlocal`, except the affected scope is the nonlocal scope.

In [12]:
x = 0
def fn1():
    x = 1
    def fn2():
        x = 2
        def fn3():
            nonlocal x
            x = 3
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn3: x = 3
inside fn2: x = 3
inside fn1: x = 1
inside global scope: x = 0


One interesting thing with the `nonlocal` keyword is that they can be chained:

In [24]:
x = 0
def fn1():
    x = 1
    def fn2():
        nonlocal x
        def fn3():
            nonlocal x
            x = 3
            print(f'inside fn3: {x = }')
        fn3()
        print(f'inside fn2: {x = }')
    fn2()
    print(f'inside fn1: {x = }')
fn1()
print(f'inside global scope: {x = }')

inside fn3: x = 3
inside fn2: x = 3
inside fn1: x = 3
inside global scope: x = 0


## Functions `globals` and `locals`

There is another way to access global namespace from within a lower-level scope: The built-in function `globals`. Its documentation speaks for itself.

In [ ]:
help('globals')

Help on built-in function globals in module builtins:

globals()
    Return the dictionary containing the current scope's global variables.

    NOTE: Updates to this dictionary *will* affect name lookups in the current
    global scope and vice-versa.



Its sibling, `locals`, is similar but we don't have the guarantee that we can use it to change local variables. But why would we want that? We can simply change local variables in the usual way. I mean, we don't have to do `locals()['x'] = 1`, we can simply write `x = 1`. (We don't have this direct option for the global variables, which is why we need the `global` keyword.)

In [26]:
help('locals')

Help on built-in function locals in module builtins:

locals()
    Return a dictionary containing the current scope's local variables.

    NOTE: Whether or not updates to this dictionary will affect name lookups in
    the local scope and vice-versa is *implementation dependent* and not
    covered by any backwards compatibility guarantees.

